# Tác vụ 3: Sử dụng Amazon Bedrock để trả lời câu hỏi

Trong sổ tay này, bạn sẽ học cách sử dụng mô hình Bedrock Titan để cung cấp thông tin phản hồi cho các truy vấn thông qua việc gửi yêu cầu có đầy đủ ngữ cảnh liên quan đến mô hình và chờ nhận phản hồi, giải quyết thách thức khi mô hình trả về câu trả lời thực tế cho các câu hỏi mà không cần chuẩn bị và lập chỉ mục tài liệu từ trước.

Sổ tay này mô phỏng những gì mà phương thức **Tạo tăng cường truy xuất (RAG)** sẽ thực hiện, nhưng không thực sự sử dụng RAG. Cách tiếp cận này sẽ phát huy hiệu quả với các tài liệu ngắn hoặc ứng dụng single-ton (đơn lẻ), nhưng lại không có khả năng thay đổi quy mô để trả lời các câu hỏi cấp doanh nghiệp do không thể đưa hết các tài liệu doanh nghiệp lớn vào câu lệnh gửi đến mô hình.

**Trả lời câu hỏi (QA)** là một nhiệm vụ quan trọng liên quan đến việc trích xuất câu trả lời cho các truy vấn thực tế được đặt ra bằng ngôn ngữ tự nhiên. Thông thường, hệ thống QA xử lý truy vấn dựa trên cơ sở kiến thức chứa dữ liệu có cấu trúc hoặc phi cấu trúc và tạo ra phản hồi có thông tin chính xác. Bảo đảm độ chính xác cao là điểm mấu chốt để phát triển một hệ thống trả lời câu hỏi hữu ích, ổn định và đáng tin cậy, nhất là cho các trường hợp sử dụng của doanh nghiệp.


## Kịch bản

Bạn thử lập mô hình cho một tình huống tại AnyCompany. Ở tình huống này, bạn sẽ yêu cầu một mô hình trả lời câu hỏi cung cấp thông tin về việc thay lốp cho một mẫu xe cụ thể mà họ sản xuất. Trước tiên, bạn truy vấn mô hình bằng phương pháp "zero shot" (không cần mẫu) để xem mô hình có thể cung cấp câu trả lời phù hợp mà chỉ cần dựa trên dữ liệu đào tạo sẵn có hay không.

Tuy nhiên, bạn nhận ra rằng mô hình này dường như có “ảo giác” với những câu trả lời chung chung hơn. Bằng chứng là khi bạn thử một mô hình xe giả thì vẫn nhận được những câu trả lời tương tự. Điều này cho thấy cần phải tăng cường đào tạo mô hình bằng các hướng dẫn sử dụng xe thực tế của Công ty ví dụ để cung cấp thông tin chi tiết về lốp xe của từng mẫu xe.

Trong bài thực hành này, bạn sẽ mô phỏng phương pháp "Tạo tăng cường truy xuất"(RAG) như vậy mà không cần dữ liệu bên ngoài. Bạn cung cấp một đoạn trích trong tài liệu hướng dẫn chi tiết giải thích cách thay lốp cho xe Model Z của AnyCompany. Bạn kiểm tra xem lúc này mô hình có thể đưa ra câu trả lời chính xác, tùy chỉnh dựa trên nội dung được cung cấp hay không.

## Tác vụ 3.1: Thiết lập môi trường

Trong tác vụ này, bạn sẽ thiết lập môi trường của mình.

In [ ]:
#ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3
import botocore

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))



## Tác vụ 3.2: Hỏi đáp với kiến thức của mô hình
Trong phần này, chúng ta sẽ cố gắng sử dụng một mô hình mà dịch vụ Bedrock cung cấp để trả lời các câu hỏi dựa trên kiến ​​thức mà mô hình thu được trong giai đoạn đào tạo.

Trong tác vụ này, bạn sử dụng phương thức invoke_model() của ứng dụng Amazon Bedrock. Các tham số bắt buộc để sử dụng phương thức này là modelId (biểu thị ARN của mô hình Amazon Bedrock) và body (câu lệnh cho tác vụ của bạn).

Câu lệnh body sẽ thay đổi tùy thuộc vào nhà cung cấp mô hình nền tảng mà bạn chọn. Bạn sẽ thực hiện theo hướng dẫn chi tiết dưới đây.

```json
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```

Bạn cố gắng sử dụng các mô hình mà dịch vụ Bedrock cung cấp để trả lời các câu hỏi dựa trên kiến thức mà mô hình thu được trong giai đoạn đào tạo.

In [ ]:
prompt_data = """You are an helpful assistant. Answer questions in a concise way. If you are unsure about the
answer say 'I am unsure'

Question: How can I fix a flat tire on my AnyCompany AC8?
Answer:"""
parameters = {
    "maxTokenCount":512,
    "stopSequences":[],
    "temperature":0,
    "topP":0.9
    }

## Tác vụ 3.3: Gọi mô hình bằng cách chuyển nội dung JSON để tạo phản hồi

In [ ]:
#model configuration
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"
try:
    
    response = bedrock_client.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())
    answer = response_body.get("results")[0].get("outputText")
    print(answer.strip())

except botocore.exceptions.ClientError as error:
    if  error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Message']}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error


Mô hình đưa ra câu trả lời trình bày quá trình thay lốp xe bị xẹp. Tuy nhiên, nội dung giải thích tương tự có thể áp dụng cho bất kỳ chiếc xe nào. Rất tiếc, đây không phải là câu trả lời phù hợp cho mẫu AC8 của AnyCompany vì loại xe này không có lốp dự phòng. Điều này xảy ra là vì mô hình đã được đào tạo dựa trên dữ liệu có chứa hướng dẫn về cách thay lốp xe.

Một ví dụ khác về vấn đề này là khi cố gắng đặt câu hỏi giống nhau cho một thương hiệu và mẫu xe hoàn toàn không có thật, chẳng hạn như Amazon Tirana.

In [ ]:
prompt_data = "How can I fix a flat tire on my Amazon Tirana?"
body = json.dumps({"inputText": prompt_data, 
                   "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = bedrock_client.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print(answer.strip())

Với câu lệnh câu hỏi này, mô hình không thể đưa ra một câu trả lời thực tế.

Để khắc phục vấn đề này và giúp mô hình đưa ra câu trả lời dựa trên các hướng dẫn cụ thể áp dụng cho mẫu xe của bạn, bạn có thể tăng cường kiến ​​thức của mô hình ngay lập tức bằng cách cung cấp cơ sở kiến ​​thức bổ sung trong câu lệnh.

Hãy xem bạn có thể áp dụng điều này để cải thiện ứng dụng của mình như thế nào.

Giả sử sau đây là một đoạn trích trong tài liệu hướng dẫn sử dụng mẫu AC8 của AnyCompany (đây không phải là tài liệu hướng dẫn có thật, nhưng hãy giả sử là vậy). Tài liệu này cũng đủ ngắn để có thể vừa vặn trong cửa sổ ngữ cảnh Titan Large.

```plain
Lốp xe và áp suất lốp:

Lốp xe được làm bằng cao su đen và được lắp vào bánh xe trên xe bạn. Lốp xe cung cấp độ bám cần thiết khi lái xe, vào cua và phanh. Hai yếu tố quan trọng cần xem xét là áp suất lốp và độ mòn lốp vì chúng có thể ảnh hưởng đến hiệu suất và khả năng điều khiển của xe.

Nơi tìm áp suất lốp được đề xuất:

Bạn có thể tìm thấy thông số áp suất lốp được đề xuất trên nhãn áp suất bơm nằm ở trụ B bên ghế lái của xe. Ngoài ra, bạn có thể tham khảo tài liệu hướng dẫn sử dụng xe để biết thông tin này. Áp suất lốp được đề xuất có thể thay đổi tùy thuộc vào tốc độ và số lượng người ngồi hoặc tải trọng tối đa trong xe.

Bơm lại lốp xe:

Chỉ thực hiện kiểm tra áp suất lốp khi lốp xe nguội. Điều này có nghĩa là phải để xe nghỉ ít nhất ba giờ để đảm bảo nhiệt độ lốp xe bằng với nhiệt độ môi trường.

Cách bơm lại lốp xe:

    Kiểm tra áp suất lốp được đề xuất cho xe của bạn.
    Thực hiện theo hướng dẫn trên máy bơm hơi và bơm lốp đến áp suất chính xác.
    Trên màn hình trung tâm của xe, mở ứng dụng "Car status" (Trạng thái xe).
    Chuyển đến thẻ "Tire pressure" (Áp suất lốp).
    Nhấn vào tùy chọn "Calibrate pressure" (Hiệu chỉnh áp suất) và xác nhận hành động.
    Lái xe trong vài phút ở tốc độ trên 30 km/h để hiệu chỉnh áp suất lốp.

Lưu ý: Trong một số trường hợp, bạn có thể cần lái xe hơn 15 phút để loại bỏ hết các biểu tượng hoặc thông báo cảnh báo liên quan đến áp suất lốp. Nếu cảnh báo vẫn còn, hãy chờ lốp xe nguội rồi lặp lại các bước trên.

Xịt lốp:

Nếu bị xịt lốp khi đang lái xe, bạn có thể tạm thời vá lỗ thủng và bơm lại lốp bằng bộ dụng cụ vá lốp di động. Bộ dụng cụ này thường được cất dưới lớp lót của khoang hành lý trên xe.

Hướng dẫn sử dụng Bộ dụng cụ vá lốp di động:

    Mở cửa sau hoặc cốp xe.
    Nhấc lớp lót của khoang hành lý lên để lấy bộ dụng cụ vá lốp di động.
    Thực hiện theo hướng dẫn đi kèm bộ dụng cụ để vá vết thủng trên lốp.
    Sau khi sử dụng, hãy nhớ đặt bộ dụng cụ trở lại vị trí ban đầu một cách chắc chắn.
    Liên hệ với Rivesla hoặc dịch vụ phù hợp để được hỗ trợ xử lý và thay thế chai keo đã qua sử dụng.

Xin lưu ý rằng bộ dụng cụ vá lốp di động chỉ là giải pháp tạm thời. Sau khi vá, bạn chỉ có thể lái thêm tối đa 10 phút hoặc 8 km (tùy theo điều kiện nào đạt được trước) với tốc độ tối đa 80 km/h. Bạn nên thay lốp xe bị thủng hoặc gọi thợ sửa chữa chuyên nghiệp sớm nhất có thể.
```

In [ ]:
context = """Tires and tire pressure:

Tires are made of black rubber and are mounted on the wheels of your vehicle. They provide the necessary grip for driving, cornering, and braking. Two important factors to consider are tire pressure and tire wear, as they can affect the performance and handling of your car.

Where to find recommended tire pressure:

You can find the recommended tire pressure specifications on the inflation label located on the driver's side B-pillar of your vehicle. Alternatively, you can refer to your vehicle's manual for this information. The recommended tire pressure may vary depending on the speed and the number of occupants or maximum load in the vehicle.

Reinflating the tires:

When checking tire pressure, it is important to do so when the tires are cold. This means allowing the vehicle to sit for at least three hours to ensure the tires are at the same temperature as the ambient temperature.

To reinflate the tires:

    Check the recommended tire pressure for your vehicle.
    Follow the instructions provided on the air pump and inflate the tire(s) to the correct pressure.
    In the center display of your vehicle, open the "Car status" app.
    Navigate to the "Tire pressure" tab.
    Press the "Calibrate pressure" option and confirm the action.
    Drive the car for a few minutes at a speed above 30 km/h to calibrate the tire pressure.

Note: In some cases, it may be necessary to drive for more than 15 minutes to clear any warning symbols or messages related to tire pressure. If the warnings persist, allow the tires to cool down and repeat the above steps.

Flat Tire:

If you encounter a flat tire while driving, you can temporarily seal the puncture and reinflate the tire using a tire mobility kit. This kit is typically stored under the lining of the luggage area in your vehicle.

Instructions for using the tire mobility kit:

    Open the tailgate or trunk of your vehicle.
    Lift up the lining of the luggage area to access the tire mobility kit.
    Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
    After using the kit, make sure to securely put it back in its original location.
    Contact AnyCompany or an appropriate service for assistance with disposing of and replacing the used sealant bottle.

Please note that the tire mobility kit is a temporary solution and is designed to allow you to drive for a maximum of 10 minutes or 8 km (whichever comes first) at a maximum speed of 80 km/h. It is advisable to replace the punctured tire or have it repaired by a professional as soon as possible."""

##### Bây giờ, hãy chuyển toàn bộ đoạn trích sang mô hình cùng với câu hỏi.

In [ ]:
question = "How can I fix a flat tire on my AnyCompany AC8?"
prompt_data = f"""Answer the question based only on the information provided between ## and give step by step guide.
#
{context}
#

Question: {question}
Answer:"""

### Tác vụ 3.4: Gọi mô hình thông qua boto3 để tạo phản hồi

In [ ]:
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = bedrock_client.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print(answer.strip())

Vì mô hình phải mất một thời gian để hiểu ngữ cảnh và tạo ra câu trả lời phù hợp cho bạn nên điều này có thể dẫn đến trải nghiệm người dùng kém vì họ phải chờ phản hồi trong vài giây.

Bedrock cũng hỗ trợ khả năng truyền trực tuyến cho phép dịch vụ tạo đầu ra khi mô hình tạo mã token. Sau đây là ví dụ về cách thực hiện.

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
# response with stream
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

Phản hồi cung cấp các hướng dẫn tóm tắt từng bước về quy trình thay lốp xe. 

Vậy là bạn đã học được cách tận dụng quy trình Tạo tăng cường truy xuất (RAG) hoặc Tăng cường để tạo phản hồi có chọn lọc, phù hợp với ngữ cảnh và thông tin cụ thể được cung cấp.

### Tự thực hiện
- Thay đổi câu lệnh theo trường hợp sử dụng cụ thể của bạn và đánh giá đầu ra của các mô hình khác nhau.
- Thử nghiệm với độ dài mã token để nắm được độ trễ và khả năng phản hồi của dịch vụ.
- Áp dụng các nguyên tắc tạo câu lệnh khác nhau để có được đầu ra tốt hơn.

### Dọn dẹp

Bạn đã hoàn thành sổ tay này. Để chuyển sang phần tiếp theo của phòng thực hành, hãy làm như sau:

- Đóng tệp sổ tay này và tiếp tục với **Tác vụ 4**.